In [1]:
from sqlalchemy import create_engine
import pandas as pd
import yaml
import numpy as np
import sklearn
import os
from os.path import isfile

In [2]:
fb_df = pd.read_json("data/products_table.json")

In [ ]:
fb_df.head()
#8091 rows and 8 columns

In [ ]:
fb_df.info()

In [ ]:
fb_df['product_name'].describe()

In [ ]:
fb_df.columns

In [ ]:
import seaborn as sns
sns.pairplot(fb_df)

In [3]:
def remove_n_a_records(df, column: str):
    """
    Scan the column for records with all N/As. Get rid of them

    Args:
        column (str): The column currently being scanned.
    """
    # Swap N/A for the pandas nan, so we can drop them
    temp_df = df[column].replace('N/A', np.nan)
    temp_df = temp_df.dropna()
    # Create a new df with only the records without the nans
    clean_df = pd.merge(temp_df, df,
                        left_index=True, right_index=True)
    # The merge creates a duplicate column. Remove it.
    clean_df.drop(column + '_x', inplace=True, axis=1)
    # Rename the remaining category column
    clean_df.rename(columns={column + '_y': column}, inplace=True)
    # Commit the cleansed data to the dataframe
    df = clean_df
    return df

fb_df = remove_n_a_records(fb_df, 'category')

In [ ]:
fb_df

In [ ]:
#seperate numerical columns and select numerical columns
df_numeric = fb_df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

In [ ]:
#select non-numeric columns
df_non_numeric = fb_df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values

In [4]:
fb_df['price'] = fb_df['price'].apply(
            lambda x: x.strip("£").replace(',',''))
fb_df['price'] = fb_df['price'].astype('float64')

In [ ]:
#generate hierarchy from product_name, product_description and location
fb_df

In [ ]:
fb_df['price'].head(10)

In [ ]:
fb_df

In [ ]:
#check for outliers
fb_df['price'].describe()

In [ ]:
fb_df['price'].plot(kind='box', figsize=(12, 8))

In [5]:
def remove_price_outliers(df):
    
    df = df[df['price'] < 10000]
    df = df[df['price'] > 0.1]
    return df

fb_df = remove_price_outliers(fb_df)

In [ ]:
fb_df['price'].plot(kind='box', figsize=(12, 8))

In [ ]:
fb_df.describe()

In [6]:
#split categories into main using lambda
fb_df['main_category'] = fb_df['category'].apply(
    lambda x: x.split("/")[0].strip())
fb_df['sub_category'] = fb_df['category'].apply(
    lambda x: x.split("/")[1].strip())
#fb_df['mini_category'] = fb_df['category'].apply(
 #   lambda x: x.split("/")[2].strip())


In [ ]:
fb_df

In [ ]:
import seaborn as sns

In [ ]:
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geocoder = Nominatim(user_agent='GetLoc')
geocode = RateLimiter(geocoder.geocode, min_delay_seconds=0.3, return_value_on_exception=None)
#location = geocode.reverse(, timeout=10, language='en')
fb_df['geo_location'] = fb_df['location'].apply(geocode)
fb_df['longiude_latitude'] = fb_df['geo_location'].apply(
    lambda loc: tuple(loc.point) if loc else None)



In [ ]:
fb_df

In [7]:
#clean random characters from columns
fb_df['product_name'] = fb_df['product_name'].str.lower().replace('[^0-9a-zA-Z]+', '_', regex=True)
category_encodings = pd.get_dummies(
            fb_df['product_name'], prefix='product_name', drop_first=True)
fb_df['product_description'] = fb_df['product_description'].str.lower().replace('[^0-9a-zA-Z]+', '_', regex=True)
category_encodings = pd.get_dummies(
            fb_df['product_description'], prefix='product_description', drop_first=True)
fb_df['main_cateogry'] = fb_df['main_category'].str.lower().replace('[^0-9a-zA-Z]+', '_', regex=True)
category_encodings = pd.get_dummies(
            fb_df['main_category'], prefix='main_category', drop_first=True)
fb_df = pd.concat(
            [fb_df, category_encodings], axis=1)


In [8]:
fb_df

,id,product_name,category,product_description,price,location,page_id,create_time,main_category,sub_category,...,main_category_Computers & Software,main_category_DIY Tools & Materials,main_category_Health & Beauty,main_category_Home & Garden,"main_category_Music, Films, Books & Games",main_category_Office Furniture & Equipment,main_category_Other Goods,"main_category_Phones, Mobile Phones & Telecoms","main_category_Sports, Leisure & Travel",main_category_Video Games & Consoles
1,243809c0-9cfc-4486-ad12-3b7a16605ba9,mirror_wall_art_in_wokingham_berkshire_gumtree,"Home & Garden / Dining, Living Room Furniture ...",mirror_wall_art_posted_by_nisha_in_dining_livi...,5.0,"Wokingham, Berkshire",1426704584,2022-02-26,Home & Garden,"Dining, Living Room Furniture",...,0,0,0,1,0,0,0,0,0,0
2,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,stainless_steel_food_steamer_in_inverness_high...,Home & Garden / Other Household Goods,morphy_richard_s_model_no_48755_stainless_stee...,20.0,"Inverness, Highland",1426704579,2022-02-26,Home & Garden,Other Household Goods,...,0,0,0,1,0,0,0,0,0,0
3,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,sun_loungers_in_skegness_lincolnshire_gumtree,Home & Garden / Garden & Patio / Outdoor Setti...,i_have_2_of_these_collection_only_as_i_don_t_d...,20.0,"Skegness, Lincolnshire",1426704576,2022-02-26,Home & Garden,Garden & Patio,...,0,0,0,1,0,0,0,0,0,0
4,59948726-29be-4b35-ade5-bb2fd7331856,coffee_side_table_from_ammunition_ammo_box_hai...,"Home & Garden / Dining, Living Room Furniture ...",great_reclaimed_army_ammunition_box_used_as_co...,115.0,"Radstock, Somerset",1426704575,2022-02-26,Home & Garden,"Dining, Living Room Furniture",...,0,0,0,1,0,0,0,0,0,0
5,16dbc860-696e-4cda-93f6-4dd4926573fb,modern_shannon_sofa_for_sale_at_low_cost_in_de...,"Home & Garden / Dining, Living Room Furniture ...",new_design_shannon_corner_sofa_5_seater_availa...,450.0,"Delph, Manchester",1426704570,2022-02-26,Home & Garden,"Dining, Living Room Furniture",...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8085,c4148656-78a9-4f3e-b393-134fdc5ef900,sony_playstation_vr_move_bundle_in_acocks_gree...,Video Games & Consoles / Consoles / PS4 (Sony ...,sony_playstation_vr_move_bundle353cash_on_coll...,260.0,"Acocks Green, West Midlands",1422159237,2022-02-28,Video Games & Consoles,Consoles,...,0,0,0,0,0,0,0,0,0,1
8086,564e3411-768d-4250-a624-b119d696f103,playstation_vr_v2_bundle_in_acocks_green_west_...,Video Games & Consoles / Consoles / PS4 (Sony ...,playstation_vr_v2_bundle355cash_on_collection_...,235.0,"Acocks Green, West Midlands",1422159464,2022-02-28,Video Games & Consoles,Consoles,...,0,0,0,0,0,0,0,0,0,1
8088,2b0a652b-46a2-4297-b619-5efeeb222787,oculus_quest_2_256gb_in_montrose_angus_gumtree,Video Games & Consoles / Other Video Games & C...,pick_up_only_250comes_with_two_pistols_stocks_...,250.0,"Montrose, Angus",1426668818,2022-02-28,Video Games & Consoles,Other Video Games & Consoles,...,0,0,0,0,0,0,0,0,0,1
8089,719fd40a-870e-4144-b324-55dff2e66fb4,logitech_driving_force_shifter_in_carrickfergu...,Video Games & Consoles / Video Game Accessorie...,bought_at_christmas_from_currys_retailing_at_4...,30.0,"Carrickfergus, County Antrim",1426699715,2022-02-28,Video Games & Consoles,Video Game Accessories,...,0,0,0,0,0,0,0,0,0,1


In [9]:
fb_df.columns

Index(['id', 'product_name', 'category', 'product_description', 'price',
       'location', 'page_id', 'create_time', 'main_category', 'sub_category',
       'main_cateogry', 'main_category_Baby & Kids Stuff',
       'main_category_Clothes, Footwear & Accessories',
       'main_category_Computers & Software',
       'main_category_DIY Tools & Materials', 'main_category_Health & Beauty',
       'main_category_Home & Garden',
       'main_category_Music, Films, Books & Games',
       'main_category_Office Furniture & Equipment',
       'main_category_Other Goods',
       'main_category_Phones, Mobile Phones & Telecoms',
       'main_category_Sports, Leisure & Travel',
       'main_category_Video Games & Consoles'],
      dtype='object')

In [ ]:
sns.heatmap(fb_df.corr(), annot=True)

In [ ]:
#remove duplicates
columns = ['product_name', 'product_description', 'location', 'category']
fb_df.drop_duplicates(subset=columns, keep="first", )

In [13]:
X = fb_df[['main_category_Baby & Kids Stuff',
       'main_category_Clothes, Footwear & Accessories',
       'main_category_Computers & Software',
       'main_category_DIY Tools & Materials', 'main_category_Health & Beauty',
       'main_category_Home & Garden',
       'main_category_Music, Films, Books & Games',
       'main_category_Office Furniture & Equipment',
       'main_category_Other Goods',
       'main_category_Phones, Mobile Phones & Telecoms',
       'main_category_Sports, Leisure & Travel',
       'main_category_Video Games & Consoles']]
y = fb_df['price']

X

,main_category_Baby & Kids Stuff,"main_category_Clothes, Footwear & Accessories",main_category_Computers & Software,main_category_DIY Tools & Materials,main_category_Health & Beauty,main_category_Home & Garden,"main_category_Music, Films, Books & Games",main_category_Office Furniture & Equipment,main_category_Other Goods,"main_category_Phones, Mobile Phones & Telecoms","main_category_Sports, Leisure & Travel",main_category_Video Games & Consoles
1,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8085,0,0,0,0,0,0,0,0,0,0,0,1
8086,0,0,0,0,0,0,0,0,0,0,0,1
8088,0,0,0,0,0,0,0,0,0,0,0,1
8089,0,0,0,0,0,0,0,0,0,0,0,1


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1
)
X_train

,main_category_Baby & Kids Stuff,"main_category_Clothes, Footwear & Accessories",main_category_Computers & Software,main_category_DIY Tools & Materials,main_category_Health & Beauty,main_category_Home & Garden,"main_category_Music, Films, Books & Games",main_category_Office Furniture & Equipment,main_category_Other Goods,"main_category_Phones, Mobile Phones & Telecoms","main_category_Sports, Leisure & Travel",main_category_Video Games & Consoles
3039,0,0,0,1,0,0,0,0,0,0,0,0
7922,0,0,0,0,0,0,0,0,0,0,0,1
6593,0,0,0,0,1,0,0,0,0,0,0,0
3340,0,0,0,1,0,0,0,0,0,0,0,0
7085,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1055,0,0,0,0,0,0,1,0,0,0,0,0
5852,0,0,0,0,0,0,0,0,0,1,0,0
4443,0,0,0,0,0,0,1,0,0,0,0,0
271,0,0,0,0,0,1,0,0,0,0,0,0


In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)

# The coefficients
print("Coefficients: \n", lm.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [-104.57671838   10.74096287  131.5889105   -36.07710346   87.42438038
   -2.881046   -112.52316758  307.02977741   10.65899614   78.75859785
  103.24125549   -9.51117143]
Mean squared error: 330211.42
Coefficient of determination: 0.03


In [ ]:
fb_df

In [ ]:
#export cleaned data to csv file
fb_df.to_csv('data/clened_data.csv', index=False)

In [ ]:
#initialise 
#create dictionary of categories
#create dataframe of image paths
#numberies cats
#prepare image cat datapoint
